In [5]:
import gradio as gr
import pickle
import numpy as np
import cv2
from PIL import Image
from ultralytics import YOLO
from tensorflow.keras.preprocessing.image import img_to_array

# Load the disease classification model
with open('BEANS_health.pkl', 'rb') as file:
    disease_model = pickle.load(file)

# Load the YOLO plant detection model
plant_model = YOLO('Downloads/best (23).pt') 

# Define the binary class labels for the disease classification
class_labels = ["healthy", "unhealthy"]

# Function to preprocess image for disease classification
def preprocess_image(image):
    image = np.array(image)
    image = cv2.resize(image, (224, 224))  
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image.astype('float32') / 255.0  
    return image

# Function to classify plant health (disease classification)
def predict_health(image):
    preprocessed_image = preprocess_image(image)
    prediction = disease_model.predict(preprocessed_image)
    predicted_index = np.argmax(prediction, axis=1)[0]
    predicted_class = class_labels[predicted_index]
    return f"Predicted class: {predicted_class}"

# Function to detect plants and draw red bounding boxes (plant detection)
def detect_plant(image):
    results = plant_model(image)  # YOLO processes the image directly
    image_np = np.array(image)
    
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding box coordinates
        labels = result.boxes.cls.cpu().numpy()  # Get the predicted class indices
        confs = result.boxes.conf.cpu().numpy()  # Get confidence scores
        
        for box, label, conf in zip(boxes, labels, confs):
            xmin, ymin, xmax, ymax = map(int, box)
            # Draw a red bounding box with thicker lines
            cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4)  # Red color, thickness 4
            class_name = plant_model.names[int(label)]  # Get the class name
            text = f"{class_name} {conf:.2f}"
            # Put class label and confidence score above the bounding box
            cv2.putText(image_np, text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Convert the result back to a PIL image for display in Gradio
    output_image = Image.fromarray(image_np)
    return output_image

# Main function to switch between models based on user input
def model_selector(image, choice):
    if choice == "Health Condition":
        return predict_health(image), None  # Return prediction text and no image
    elif choice == "Plant Detection":
        return None, detect_plant(image)  # Return no text but output image

# Create the Gradio interface with a dropdown to select between the two models
interface = gr.Interface(
    fn=model_selector,
    inputs=[gr.Image(type="pil"), gr.Radio(["Health Condition", "Plant Detection"], label="Select Model")],
    outputs=[gr.Textbox(label="Health Prediction"), gr.Image(label="Plant Detection")],
    title="""FCMB AGRITECH INNOVATION HACKATHON.                                      
    AgriAI: AI-Powered Solution for Detecting Plants and their Health Condition"""
)

interface.launch(share=True)


Running on local URL:  http://127.0.0.1:7866

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step

0: 640x480 5 SESAME_PLANTs, 4444.2ms
Speed: 502.9ms preprocess, 4444.2ms inference, 605.1ms postprocess per image at shape (1, 3, 640, 480)
